In [3]:
!pip install sklearn datasets transformers pandas matplotlib  imblearn numpy 
!pip3 install --upgrade tensorflow-gpu
!pip install pycm

     |████████████████████████████████| 298 kB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 243 kB 70.1 MB/s 
     |████████████████████████████████| 132 kB 49.5 MB/s 
     |████████████████████████████████| 61 kB 432 kB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 192 kB 49.3 MB/s 
     |████████████████████████████████| 271 kB 67.2 MB/s 
     |████████████████████████████████| 160 kB 54.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 489.6 MB 18 kB/s 
     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 574 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import csv
# from imblearn import fit_resample
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import random
from transformers import AutoModelForSequenceClassification
from datasets import list_datasets, load_dataset, list_metrics, load_metric
import torch



In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer


raw_datasets = load_dataset('poleval2019_cyberbullying','task01')
class_names = ["non-harmful","harmful"]


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]
# full_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10)) 
# full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10)) 


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

training_args = TrainingArguments("test_trainer")

Reusing dataset poleval2019_cyber_bullying (/home/x/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [5]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 10041
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3768


Step,Training Loss


KeyboardInterrupt: 

In [12]:
checkpoint = torch.load('test_trainer/checkpoint-3500/pytorch_model.bin')
model.load_state_dict(checkpoint)


<All keys matched successfully>

In [5]:
sequence_0 = "@anonymized_account Adrian Juda, figurant WSI i  lobby żydowskiego"

def predict(sequence_0):
  paraphrase = tokenizer(sequence_0, return_tensors="pt")
  paraphrase_classification_logits = model(**paraphrase).logits
  return torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]


def predict1(sequence_0):
  paraphrase_results = predict(sequence_0)
  return int(round(paraphrase_results[1]))

class_names[predict1(sequence_0)]

NameError: ignored

In [47]:
def print_metrics(test_y,y_pred):
    print(f'f1_score={metrics.f1_score(test_y,y_pred)}')
    val = metrics.f1_score(test_y,y_pred,average='micro')
    print(f'f1_score micro={val}')
    val =metrics.f1_score(test_y,y_pred,average='macro')
    print(f'f1_score macro={val}')
    print(f'MCC={metrics.matthews_corrcoef(test_y, y_pred)}')
    print(metrics.confusion_matrix(test_y,y_pred))

In [55]:
def get_cf_values(y_test,y_pred):
  confusion_matrix = metrics.confusion_matrix(y_test,y_pred)

  for true,predict in zip(y_test,y_pred):
    if true == pred and 

  unq = np.array([x + 2*y for x, y in zip(y_test,y_pred) ])
  tp = np.array(np.where(unq == 3)).tolist()[0]
  fp = np.array(np.where(unq == 1)).tolist()[0]
  tn = np.array(np.where(unq == 0)).tolist()[0]
  fn = np.array(np.where(unq == 2)).tolist()[0]
  return [tp,fp,tn,fn]


In [2]:
# !pip install pycm
from pycm import *
cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred)
cm.position[0]['FN']

ModuleNotFoundError: ignored

In [62]:
[tp,fp,tn,fn] = get_cf_values(y_test,y_pred)
fn

[24,
 31,
 37,
 51,
 57,
 58,
 61,
 64,
 65,
 83,
 85,
 86,
 89,
 101,
 109,
 128,
 133,
 134,
 142,
 164,
 166,
 171,
 183,
 187,
 192,
 195,
 197,
 208,
 215,
 218,
 225,
 226,
 227,
 230,
 239,
 243,
 248,
 251,
 252,
 256,
 260,
 263,
 301,
 317,
 332,
 333,
 339,
 353,
 358,
 362,
 386,
 388,
 397,
 409,
 423,
 436,
 438,
 439,
 440,
 445,
 450,
 453,
 456,
 457,
 462,
 465,
 467,
 468,
 473,
 475,
 482,
 484,
 488,
 490,
 503,
 507,
 526,
 538,
 545,
 546,
 550,
 553,
 555,
 558,
 563,
 585,
 588,
 589,
 632,
 643,
 647,
 652,
 653,
 668,
 675,
 710,
 732,
 733,
 736,
 746,
 748,
 759,
 761,
 773,
 782,
 784,
 795,
 800,
 805,
 828,
 836,
 840,
 842,
 852,
 859,
 860,
 890,
 904,
 911,
 930,
 931,
 933,
 938,
 941,
 961,
 963,
 973,
 976,
 977,
 983,
 984,
 985,
 993,
 998]

In [44]:
y_test = full_eval_dataset['label']
X_test = full_eval_dataset['text']
y_pred = [predict1(statement) for statement in X_test]

In [48]:
print_metrics(y_test,y_pred)

f1_score=0.0
f1_score micro=0.866
f1_score macro=0.4640943193997856
MCC=0.0
[[866   0]
 [134   0]]


In [15]:
input_text = "The text on which I test"
input_text_tokenized = tokenizer.encode(input_text,
                                        truncation=True,
                                        padding=True,
                                        return_tensors="tf")
prediction = model(input_text_tokenized)
prediction_logits = prediction[0]
prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
print(f'The prediction probs are: {prediction_probs}')


AttributeError: ignored

In [10]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

compute_metrics(eval_pred)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


{'eval_accuracy': 0.7,
 'eval_loss': 0.5977295637130737,
 'eval_runtime': 0.7741,
 'eval_samples_per_second': 12.918,
 'eval_steps_per_second': 2.584}